In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 《室用途別の床面積に応じた用途別面積の推定》

#### 室用途別面積比率
$$ \qquad
r_{主}= \frac{A_主}{A_主 + A_他 + A_非}, \quad
r_{他}= \frac{A_他}{A_主 + A_他 + A_非}\quad, \quad
r_{非}= \frac{A_非}{A_主 + A_他 + A_非}\quad
\qquad
\\
$$
- 入力値：
  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$

- 出力値：
  - 主居室、その他の居室、非居室の外皮の面積比率$[-]$：$r_{主}$、$r_{他}$、$r_{非}$


#### 室用途別の面積および長さの推定
$$ \qquad
A_{主,i,j}= {A_{i,j}・r_主}, \quad
A_{他,i,j}= {A_{i,j}・r_他}\quad, \quad
A_{非,i,j}= {A_{i,j}・r_非}\quad
\qquad
\\
$$
$$ \qquad
L_{主,i,j}= {L_{i,j}・r_主}, \quad
L_{他,i,j}= {L_{i,j}・r_他}\quad, \quad
L_{非,i,j}= {L_{i,j}・r_非}\quad
\qquad
\\
$$
- 入力値：
  - 外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,j}$
  - 熱橋及び土間床等の方位jの外周部𝑖の長さ$[m]$：$L_{i,j}$

- 出力値：
  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{主,i,j}$、$A_{他,i,j}$、$A_{非,i,j}$
  - 主居室、その他の居室、非居室の熱橋及び土間床等の方位jの外周部𝑖の長さ$[m]$：$L_{主,i,j}$、$L_{他,i,j}$、$L_{非,i,j}$

#### Function
##### 室用途別面積比率の取得

In [2]:
def GetAreaRate( main_floor_area, other_floor_area, total_floor_area ):
    # total_floor_area as double : 床面積の合計[m]
    # main_floor_area as double  : 主たる居室の床面積[m]
    # other_floor_area as double : その他の居室の床面積[m]
    nonliving_floor_area = total_floor_area - ( main_floor_area + other_floor_area )
    return main_floor_area / total_floor_area, other_floor_area / total_floor_area, nonliving_floor_area / total_floor_area

##### 室用途別の面積の取得

In [3]:
def GetAreaByRoomUse( target_area, main_floor_area, other_floor_area, total_floor_area ):
    # target_area as double      : 方位j、部位iの面積[m^2]
    # total_floor_area as double : 床面積の合計[m]
    # main_floor_area as double  : 主たる居室の床面積[m]
    # other_floor_area as double : その他の居室の床面積[m]    
    
    rate_main, rate_other, rate_nonliving = GetAreaRate( main_floor_area, other_floor_area, total_floor_area )
    return {
        'main'      : rate_main * target_area,
        'other'     : rate_other * target_area,
        'nonliving' : rate_nonliving * target_area
    }

##### 室用途別の長さの取得

In [4]:
def GetLengthByRoomUse( target_length, main_floor_area, other_floor_area, total_floor_area ):
    # target_length as double      : 方位j、部位iの長さ[m]
    # total_floor_area as double : 床面積の合計[m]
    # main_floor_area as double  : 主たる居室の床面積[m]
    # other_floor_area as double : その他の居室の床面積[m]    
    
    rate_main, rate_other, rate_nonliving = GetAreaRate( main_floor_area, other_floor_area, total_floor_area )
    return {
        'main'      : rate_main * target_length,
        'other'     : rate_other * target_length,
        'nonliving' : rate_nonliving * target_length
    }

#### Example

In [5]:
areas = GetAreaByRoomUse(50.0, 30.0, 60.0, 120.0)
lengths = GetLengthByRoomUse(12.0, 30.0, 60.0, 120.0)
areas, lengths

({'main': 12.5, 'nonliving': 12.5, 'other': 25.0},
 {'main': 3.0, 'nonliving': 3.0, 'other': 6.0})

## 2. コンバート
### 1) 共通

- 地域の区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)、簡易外皮入力フラグの情報は維持する。

#### Input

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Output

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Function

In [6]:
def convert_common(d):
    return d['Common']

#### Example

In [7]:
d = {
    'Common': {
        'Region' : 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
}
}
convert_common(d)

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 壁体名称、一般部位構造種別、壁体種別、方位、層構成（部材名称、厚さ[m]、熱伝導率[W/mK]、容積比熱[(J/(m<sup>3</sup>・K)]）などの情報は維持する。
- Lv2の入力情報に「壁体が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [8]:
def convert_wall( d ):
    
    walls =[]
    for arg_wall in d['Walls']:
        areas = GetAreaByRoomUse(arg_wall['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
             # 壁体名称、部位面積を更新し、「壁体が属する空間種類」を追加する
            wall = copy.deepcopy(arg_wall)
            wall['name'] = arg_wall['name'] + '_' + space 
            wall['area'] = area
            wall['space'] = space
            walls.append(wall)
            
    return walls

#### Example

In [9]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Walls': [
        { 'area': 67.8, 'direction': 'top', 'name': 'Ceiling' },
        { 'area': 40.63, 'direction': 'SW', 'name': 'Wall_SW',}
    ]
}    

convert_wall(d)

[{'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main'},
 {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other'},
 {'area': 16.95,
  'direction': 'top',
  'name': 'Ceiling_nonliving',
  'space': 'nonliving'},
 {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main'},
 {'area': 20.315,
  'direction': 'SW',
  'name': 'Wall_SW_other',
  'space': 'other'},
 {'area': 10.1575,
  'direction': 'SW',
  'name': 'Wall_SW_nonliving',
  'space': 'nonliving'}]

### 3) 開口部の透明部位

- 開口部名称、方位、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [10]:
def convert_window( d ):
    
    windows =[]
    for arg_window in d['Windows']:
        areas = GetAreaByRoomUse(arg_window['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            window = copy.deepcopy(arg_window)
            window['name'] = arg_window['name'] + '_' + space
            window['area'] = area
            window['space'] = space
            windows.append(window)
        
    return windows

#### Example

In [11]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17  }
    ]
}    

convert_window(d)

[{'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main'},
 {'area': 15.125,
  'direction': 'SW',
  'name': 'WindowSW_other',
  'space': 'other'},
 {'area': 7.5625,
  'direction': 'SW',
  'name': 'WindowSW_nonliving',
  'space': 'nonliving'},
 {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main'},
 {'area': 1.585,
  'direction': 'NW',
  'name': 'WindowNW_other',
  'space': 'other'},
 {'area': 0.7925,
  'direction': 'NW',
  'name': 'WindowNW_nonliving',
  'space': 'nonliving'}]

### 4) 開口部の不透明部位

- 開口部名称、方位、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [12]:
def convert_door( d ):
    
    doors =[]
    for arg_door in d['Doors']:
        areas = GetAreaByRoomUse(arg_door['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            door = copy.deepcopy(arg_door)
            door['name'] = arg_door['name'] + '_' + space
            door['area'] = area
            door['space'] = space
            doors.append(door)
        
    return doors

#### Example

In [13]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 52 }
    ]
}    
convert_door(d)

[{'area': 13.0, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
 {'area': 26.0, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
 {'area': 13.0,
  'direction': 'NW',
  'name': 'DoorNW_nonliving',
  'space': 'nonliving'}]

### 5) 線熱橋

- 熱橋部の構造種別、方位、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「 熱橋部が属する空間種類」の情報を追加し、熱橋長さから室用途別の熱橋長さを推定する。

#### Function

In [14]:
def convert_heatbridge( d ):
    
    heatbridges =[]
    for arg_heatbridge in d['Heatbridges']:
        lengths = GetLengthByRoomUse(arg_heatbridge['length'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, length in lengths.items() :
            # 熱橋部長さを更新し、「熱橋部が属する空間種類」を追加する
            heatbridge = copy.deepcopy(arg_heatbridge)
            heatbridge['length'] = length
            heatbridge['space'] = space
            heatbridges.append(heatbridge)
        
    return heatbridges

#### Example

In [15]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Heatbridges': [
        { 'structure': 'rc', 'direction1': 'NW', 'direction2': 'NE', 'length': 10 }
    ]
}    
convert_heatbridge(d)

[{'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'main',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 5.0,
  'space': 'other',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'nonliving',
  'structure': 'rc'}]

### 6) 土間床等の外周部の線熱橋

- 方位、熱橋長さ[m]、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「土間床等の外周部の線熱橋が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [16]:
def convert_earthfloorperimeter( d ):
    
    def make_earthfloorperimeter( f ):
        earthfloor_perimeter = copy.deepcopy( f )
        earthfloor_perimeter['space'] = 'underfloor'
        return earthfloor_perimeter
    
    return [ make_earthfloorperimeter(f) for f in d['EarthfloorPerimeters'] ]

#### Example

In [17]:
d = {
    'EarthfloorPerimeters': [
        { 'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8 },
        { 'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8 }
    ]
}    
convert_earthfloorperimeter(d)

[{'direction': 'NW',
  'length': 2.43,
  'name': 'other_NW',
  'psi': 1.8,
  'space': 'underfloor'},
 {'direction': 'NE',
  'length': 2.43,
  'name': 'other_NE',
  'psi': 1.8,
  'space': 'underfloor'}]

### 7) 土間床
- 面積[m<sup>2</sup>]などの情報は維持する。
- 「土間床等が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [18]:
def convert_earthfloor( d ):
    
    def make_earthfloor( f ):
        earthfloor = copy.deepcopy( f )
        earthfloor['space'] = 'underfloor'
        return earthfloor
    
    return [ make_earthfloor(f) for f in d['Earthfloors'] ]

#### Example

In [19]:
d = {
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0 }
    ]
}    
convert_earthfloor(d)

[{'area': 5.0, 'name': 'other', 'space': 'underfloor'}]

## 2-6. コンバート【全分類】

In [14]:
def Convert(d):
    d_common=convert_common(d['Common'])
    d_ceiling=convert_wall(d['Ceiling'])
    d_wall=convert_wall(d['Wall'])
    d_floor=convert_wall(d['Floor'])
    d_window=convert_window(d['Window'])
    d_door=convert_window(d['Door'])
    d_sunshade=convert_sunshade(d['Sunshade'])

    temporary=convert_element(d_common, d['Elements'])
    d_element=[]
    for e in temporary:
        if e['name']=='Ceiling':
            e['Wall']=d_ceiling
        elif e['name']=='Wall':
            e['Wall']=d_wall
        elif e['name']=='Floor':
            e['Wall']=d_floor
        elif e['name']=='EarthFloor':
            e['FloorNonPerimeter']={
                    'name'                       : 'FloorNotPerimeter',
                    'structure'                  : 1,
                    'type'                       : 3,
                    'Layers'                     : [{'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
                    'Ua'                         : 3.77,
                    'Ug'                         : None,
                    'HeatBridge'                 : None,
                    'rHeatBridge'                : None,
                    'UHeatBridge'                : None,
                    'lHeatBridge'                : None,
                    'psiHeatBridge'              : None,
                    'Ur'                         : None
                }
        elif e['name']=='Window':
            e['Window']=d_window
            e['Sunshade']=d_sunshade
        elif e['name']=='Door':
            e['Door']=d_door
        d_element.append(e)

    return {'common':d_common,'element':d_element}

In [15]:
d = {
    'Common' : {
        'Region'                     : 6,
        'FlagSimplifiedEnvelopeInput': 2,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    },
    'Ceiling' : {
        'name'                       : 'Ceiling',
        'structure'                  : 1,
        'type'                       : 1,
        'Layers'                     : [{'material': 'GW16K','d': 0.179,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.24,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Wall' : {
        'name'                       : 'Wall',
        'structure'                  : 1,
        'type'                       : 2,
        'Layers'                     : [{'material': 'GW16K','d': 0.076,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.53,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Floor' : {
        'name'                       : 'Floor',
        'structure'                  : 1,
        'type'                       : 3,
        'Layers'                     : [{'material': 'GW16K','d': 0.081,'rambda': 0.045,'Crho': 13},
                                        {'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
        'Ua'                         : 0.48,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Window' : {
        'name'                        : 'Window',
        'eta'                         : 0.738,
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 4.65
    },
    'Door' : {
        'name'                        : 'Door',
        'eta'                         : round(2.33*0.034,3),
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 2.33
    },
    'Sunshade' : {
        'name'                        : 'Sunshade',
        'y1'                          : 0.5,
        'y2'                          : 2, 
        'z'                           : 1,
        'fCooling'                    : None,
        'fHeating'                    : None
    },
    'Elements' : [{
            'name'                    : 'Ceiling',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 1,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Wall',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Floor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 12,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'EarthFloor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 15,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Window',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 2,
            'A'                       : 6,
            'L'                       : None,
            'sunshade'                : 'Sunshade'
        },{
            'name'                    : 'Door',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 2,
            'FlagUnsteady'            : 2,
            'A'                       : 3,
            'L'                       : None,
            'sunshade'                : None
        }
    ]
}

Convert(d)

KeyError: 'IsSimplifiedInput'